# Name : Anif Saviyo Rodrigues

# Assignment no. 7

# Car Price Prediction

### Importance of Car Price Prediction
##### Car price prediction plays an important role for buyers and sellers in the automotive industry to facilitate informed decisions in terms of
##### opportunities or potential problems. It allows businesses to make decisions informed by their historical data and sales behaviour and the
##### ongoing evolution of market dynamics.

## Installing and importing necessary libraries and modules

In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

In [3]:
dataset = pd.read_csv(r"C:\Users\X280\OneDrive\Documents\UNMESSENGER INTERNSHIP\Model Selection\car data.csv")

In [4]:
print(len(dataset))
dataset.head()

301


,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


In [6]:
dataset.drop('Car_Name',axis = 1, inplace = True)

In [7]:
dataset

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...
296,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0
297,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0
298,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
299,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0


## Overview
#### Below listed are the independent variables or relevant features:
#### Year: The manufacturing year of the car.
#### Present_Price: The current market price of the car.
#### Kms_Driven: The total distance the car has been driven.
#### Fuel_Type: The type of fuel the car uses i.e. petrol, diesel or CNG.
#### Seller_Type: The type of seller i.e. dealership or an individual owner.
#### Transmission: The type of transmission i.e. manual or automatic.
#### Owner: The number of previous owners of the car.

#### (Car_Name: The name or model of the car. This feature has been elminated because it does not directly impact the car price prediction. )

#### Selling_Price: The price at which the car is being sold. This is the dependent or target variable.


# Prediction of the Car Price Using Random Forest, XG Boost and KNN Algorithms

# Task 1: Data Preprocessing:

#### The data uploaded seems to be clean without any null values or missing values.


#### There are three columns with object data type colums or categorical dat columns which has to be encoded into numerical data.

In [8]:
#Checking the value counts in the categorical columns
print(dataset.Fuel_Type.value_counts())
print(dataset.Seller_Type.value_counts())
print(dataset.Transmission.value_counts())

Petrol    239
Diesel     60
CNG         2
Name: Fuel_Type, dtype: int64
Dealer        195
Individual    106
Name: Seller_Type, dtype: int64
Manual       261
Automatic     40
Name: Transmission, dtype: int64


In [9]:
# Encoding Categorical Data
dataset.replace({'Fuel_Type':{'Petrol':0, 'Diesel':1, 'CNG':2}},inplace=True)
dataset.replace({'Seller_Type':{'Dealer':0, 'Individual':1}},inplace=True)
dataset.replace({'Transmission':{'Manual':0, 'Automatic':1}},inplace=True)

In [10]:
dataset.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,2014,3.35,5.59,27000,0,0,0,0
1,2013,4.75,9.54,43000,1,0,0,0
2,2017,7.25,9.85,6900,0,0,0,0
3,2011,2.85,4.15,5200,0,0,0,0
4,2014,4.60,6.87,42450,1,0,0,0


# Task 2: Feature Selection and Engineering:

#### The feature selection is made based on the impact on the car price prediction and correlation with the selling price.
#### As mentioned above Car Name is been eliminated as it doesnt directly impact the car price prediction.
#### Selling Price is the target variable in the dataset and that is what we are training the model to predict.
#### The older the manufacturing year the higher the depreciation rate.
#### The more the kms driven the lower the selling price.
#### The fuel type, seller type and transmission also determines the value of the car.
#### for eg.: Petrol fuel type car is the most affordable compared to diesel and cng cars.
#### Dealerships add various costs to the selling price which will result the price of the car to be higher than the car owned by an individual.
#### Automatic transmission technology is more complex and requires more parts than manual gearbox technology. This makes the automatic transmission more expensive than the manual gearbox, eventually resulting in higher prices for automatic cars.

# Task 3: Algorithms:
### I have chosen Random Forest, XG Boost and KNN regressor algorithms for my assignment

#### Random Forest: Random Forest is an ensemble technique capable of performing both regression and classification tasks with the use of multiple decision trees and a technique called Bootstrap and Aggregation, commonly known as bagging. The basic idea behind this is to combine multiple decision trees in determining the final output rather than relying on individual decision trees. Random Forest has multiple decision trees as base learning models. We randomly perform row sampling and feature sampling from the dataset forming sample datasets for every model. This part is called Bootstrap.

#### Suitability:
#### Random Forest is great for car price prediction because it handles both numerical and categorical features well. It can capture complex relationships in the data and is less prone to overfitting, which is useful when dealing with various car attributes.

#### XG Boost or Extreme Gradient Boost: XGBoost, which stands for Extreme Gradient Boosting, is a scalable, distributed gradient-boosted decision tree (GBDT) machine learning library. It provides parallel tree boosting and is the leading machine learning library for regression, classification, and ranking problems. It’s vital to an understanding of XGBoost to first grasp the machine learning concepts and algorithms that XGBoost builds upon: supervised machine learning, decision trees, ensemble learning, and gradient boosting.


#### Suitability:
#### XGBoost is suitable for car price prediction because it's highly accurate. It can capture non-linear relationships and interactions between features effectively.

#### KNN Regressor: KNN regression is a non-parametric method that, in an intuitive manner, approximates the association between independent variables and the continuous outcome by averaging the observations in the same neighbourhood.

#### Suitability:
#### KNN makes few assumptions about the data distribution, making it versatile.


# Task 3: Training Evaluation and Selection:

In [11]:
#Splitting the dataset into training and tessting data
X = dataset.drop(['Selling_Price'],axis=1)
Y = dataset['Selling_Price']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [12]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [13]:
# Define the model: Init RF Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=0)

# Fit the model
rf_regressor.fit(X_train, Y_train)

# Predict the test set results
Y_pred = rf_regressor.predict(X_test)

# Evaluate Model
rfr2 = r2_score(Y_test, Y_pred)
print("RF R-squared Score:", rfr2)

RF R-squared Score: 0.9102369798502957


In [14]:
# Define the model: Init XG Boost Regressor
xgb_regressor = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=0)

# Fit the model
xgb_regressor.fit(X_train, Y_train)

# Predict the test set results
Y_pred = xgb_regressor.predict(X_test)

# Evaluate Model
xgr2 = r2_score(Y_test, Y_pred)
print("XG R-squared Score:", xgr2)

XG R-squared Score: 0.9133849577237275


In [15]:
# Define the model: Init KNN Regressor
knn_regressor = KNeighborsRegressor(n_neighbors=11, p=2,metric='euclidean')

# Fit the model
knn_regressor.fit(X_train, Y_train)

# Predict the test set results
Y_pred = knn_regressor.predict(X_test)

# Evaluate Model
knnr2 = r2_score(Y_test, Y_pred)
print("KNN R-squared Score:", knnr2)

KNN R-squared Score: 0.8117117160070677


In [16]:
print("RF R-squared Score:", rfr2)
print("XG R-squared Score:", xgr2)
print("KNN R-squared Score:", knnr2)

RF R-squared Score: 0.9102369798502957
XG R-squared Score: 0.9133849577237275
KNN R-squared Score: 0.8117117160070677


##  XG boost is the best performing model